<a href="https://colab.research.google.com/github/chongzicbo/nlp-ml-dl-notes/blob/master/Transformer-XL%E8%AF%A6%E8%A7%A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Transformer 的缺点

* 算法无法建模超过固定长度的依赖关系
* 被分割的片段通常不考虑句子边界，导致上下文碎片化，从而导致优化低效。

## 2. Transformer XL 

&emsp;&emsp;针对transformer的缺陷，Transformer XL提出两点来进行改进，一是引入了循环机制来提升模型的长距离依赖捕获能力，二是引入上述循环机制后，之前的绝对位置就不work了，需要新的方法引入位置信息，因此提出了相对位置编码的做法。

###2.1 segment-level recurrence with state reuse

Transformer模型在训练和评估时如下图：
<img src="https://img2018.cnblogs.com/blog/1335117/201909/1335117-20190929141228458-1663969951.png" width="500">

&emsp;&emsp;在训练阶段，序列之间是相互孤立的，在评估阶段，在预测下一个词的时候也只能利用前面的固定为n个词的序列。因此训练时模型捕获信息的长度不够，且在评估时利用信息的能力也不够。

Transformer XL模型在训练和评估时如下图：

<img src="https://img2018.cnblogs.com/blog/1335117/201909/1335117-20190929141638883-2130980423.png" width="700">

&emsp;&emsp;Transformer XL在训练的时候将上一片段的结果引入到下一片段中，在评估时同样如此。因此在捕获长距离信息的能力上较Transformer有很大的提升。公式如下：

<img src="https://img2018.cnblogs.com/blog/1335117/201909/1335117-20190929143017052-1132893676.png" width="700">

&emsp;&emsp;$h_ \tau ^n$是一个L*d的矩阵，表示的是第$\tau$个输入片段的第n层的隐藏层的状态。L表示片段长度，d表示嵌入维度。SG表示Stop Gradient,主要是为了避免RNN出现的一些列问题。$\tau$表示上一片段，$\tau +1$表示下一片段。将上一片段的输出缓存起来，然后直接和下一片段的隐层拼接在一起。

&emsp;&emsp;另外，这里引入上一片段的隐层表示只会用在key和value上，对于query还是保持原来的样子。这样做也好理解，query只是表示查询的词，而key，value表示的是表示这个查询的词的相关信息，我们要改变的是只是信息，因此只要在key，value中引入上一片段的信息就可以了，剩下的就和Transformer一致。

### 2.2 相对位置编码

&emsp;&emsp;虽然引入了上述机制，但是直接应用segment-level recurrence是行不通的，因为当我们重用前面的片段时，位置编码是不一致的。例如，考虑一个具有上下文位置[0,1,2,3]的旧段。当处理一个新的段时，我们将两个段合并，得到位置[0,1,2,3,0,1,2,3]，其中每个位置id的语义在整个序列中是不连贯的。如下图：

<img src="https://img2018.cnblogs.com/blog/1335117/201909/1335117-20190929144232284-351940163.png" width="700">

用绝对 位置表示时，对于上一片段和当前片段的位置向量是一致的。为此，我们提出了一种新的相对位置编码方案，使递归机制成为可能。

Transformer 中的attention权重计算公司如下：
$$
{(E_{x_i} + U_i)}^T W_q^T W_k (E_{x_j} + U_j)
$$
将其展开可分为以下四个部分：

<img src="https://img2018.cnblogs.com/blog/1335117/201909/1335117-20190929144441128-837448604.png" width="700">

这四个部分可以理解为：

　　a）基于内容的“寻址”，即没有添加位置向量，词对词的分数。

　　b）基于内容的位置偏置，相当于当前内容的位置偏差。

　　c）全局的内容偏置，用于衡量key的重要性。

　　d）全局的位置偏置，根据key和query调整位置的重要性。


拆分成这四部分之后，我们就可以对其进行改写，引入相对位置向量。具体做法如下：

<img src="https://img2018.cnblogs.com/blog/1335117/201909/1335117-20190929145022186-1375978886.png" width="700">

a）部分基本不变，只是对于key的位置向量的权重矩阵和词向量的权重矩阵不再共享；b）部分引入了相对位置向量$R_{i-j}$，是一个不可以学习的预先给定好的正弦编码矩阵；c）对于query的位置向量采用可以学习的初始化向量来表示，$u^T$表示对key中词的位置向量，d）同上，$u^T$表示对key中位置的位置向量。

　　将上面的式子合并后，可以得到：

$$
{(W_q E_{x_i} + u)}^T W_{k, E} E_{x_j} + {(W_q E_{x_i} + v)}^T W_{k, R} R_{i-j}
$$　

总的来说，Relative Positional Encodings就是在计算attention分数时，用相对位置$R_{i-j}$编码来代替原来的绝对位置编码$U_i$和$U_j$。并且学习了相对位置v和u用来调整不同距离和不同嵌入的得分。

## 3. 总结

&emsp;&emsp;总的来说TransformerXL对Transformer进行了一些调整，试图解决一些问题。按照论文的描述，TransformerXL学习的依赖关系比RNN长80%，比传统Transformer长450%，在短序列和长序列上都获得了更好的性能，并且在评估阶段比传统Transformer快1800+倍。














